In [ ]:
#Create a trigram model. Use two chars to predict the third and evaluate the loss. Compare loss to bigram model

In [1]:
!ls

 1.Building_micrograd.ipynb   '2. Trigrams_and_exercises.ipynb'   nb.ipynb
'2. Building_Makemore.ipynb'   names.txt


In [2]:
import torch
import torch.nn.functional as F
import random

In [3]:
words = open('names.txt', 'r').read().splitlines()

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [186]:
xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        xs.append([ix1,ix2])
        ys.append(ix3)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
print(xs,ys)
num = len(xs)
print("number of examples: ", num)
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27*2, 27),generator = g, requires_grad = True)

tensor([[ 0,  5],
        [ 5, 13],
        [13, 13],
        ...,
        [26, 25],
        [25, 26],
        [26, 24]]) tensor([13, 13,  1,  ..., 26, 24,  0])
number of examples:  196113


In [187]:
xs.shape, ys.shape

(torch.Size([196113, 2]), torch.Size([196113]))

In [7]:
xenc = F.one_hot(xs, num_classes = 27).float()

In [8]:
xenc.shape, xenc.dtype

(torch.Size([196113, 2, 27]), torch.float32)

In [9]:
xenc = xenc.reshape(num, -1)

In [12]:
xenc[0]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
xenc.shape

torch.Size([196113, 54])

In [11]:
W.shape

torch.Size([54, 27])

In [239]:
for k in range(100):
    #forward pass
    xenc = F.one_hot(xs, num_classes = 27).float()
    xenc = xenc.reshape(num, -1)
    logits = xenc @ W # 196113,54 x 54,27 -> 196113,27
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01 * (W**2).mean()
    print(loss.item())

    #backward
    W.grad = None
    loss.backward()

    #update
    W.data += -((100 - k)/10) * W.grad
    #W.data += -40 * W.grad

2.2523412704467773
2.2523415088653564
2.2523412704467773
2.2523412704467773
2.2523412704467773
2.2523412704467773
2.2523412704467773
2.2523410320281982
2.2523410320281982
2.2523410320281982
2.252340793609619
2.252340793609619
2.252340793609619
2.25234055519104
2.252340793609619
2.252340793609619
2.252340793609619
2.252340793609619
2.25234055519104
2.25234055519104
2.25234055519104
2.25234055519104
2.25234055519104
2.25234055519104
2.25234055519104
2.252340316772461
2.252340316772461
2.252340316772461
2.252340316772461
2.252340078353882
2.252340078353882
2.252340078353882
2.252340078353882
2.252340078353882
2.252340078353882
2.252340078353882
2.252340078353882
2.252340078353882
2.252340078353882
2.2523398399353027
2.2523398399353027
2.2523398399353027
2.2523396015167236
2.2523396015167236
2.2523396015167236
2.2523396015167236
2.2523396015167236
2.2523396015167236
2.2523398399353027
2.2523398399353027
2.2523398399353027
2.2523398399353027
2.2523398399353027
2.2523398399353027
2.252339363

In [260]:
loss.item()

2.2523391246795654

In [288]:
#sampling from NN
g = torch.Generator().manual_seed(2147483647)

for i in range(50):
    out = []
    ix = 0
    iy = random.randint(1,26)
    while True:
        xenc = F.one_hot(torch.tensor([ix,iy]), num_classes = 27).float()
        xenc = xenc.reshape(-1, 54)
        logits = xenc @ W
        counts = logits.exp()
        p = counts / counts.sum(1,keepdim=True)
        iy = torch.multinomial(p, num_samples = 1, replacement = True, generator = g).item()
        out.append(itos[iy])
        ix = iy
        if iy == 0:
            break
    print(''.join(out))

munide.
ianasad.
u.
alay.
a.
in.
esi.
ritolia.
i.
yete.
ulinaduwinileviasededaine.
i.
olese.
y.
uly.
arelynavelethafon.
ume.
n.
nisy.
ahasani.
core.
yarigrely.
vabi.
herimiki.
wynin.
anadsn.
avolesaselhadiosemitiavili.
e.
pina.
ryaterynosli.
isa.
ube.
emjelumemahadysyaleva.
ucoaro.
ydeh.
renita.
a.
ay.
abely.
a.
i.
elelooni.
ahamiave.
ara.
u.
anely.
on.
slitiki.
saloeve.
on.
